# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from sklearn import preprocessing

In [2]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [3]:
from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : Тип метки (напр., int или str)
        Метка класса, который встречается чаще всего среди элементов листа дерева
    """
    def __init__(self,  labels):
        uniq_labels, cnts = np.unique(labels, return_counts=True)
        self.y = uniq_labels[np.argmax(cnts)]
        probs = np.array(cnts, dtype=np.float) / labels.shape[0]
        self.dict = {}
        for label, prob in zip(uniq_labels, probs):
            self.dict[label] = prob

class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value.
    """
    def __init__(self, split_dim: int, split_value: float,
                 left: Union['DecisionTreeNode', DecisionTreeLeaf],
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [4]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.root = None
        if criterion == "gini":
            self.criterion = gini
        else:
            self.criterion = entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        if max_features == "auto":
            self.max_features = np.floor(np.sqrt(X.shape[1])).astype(np.int)
        else:
            self.max_features = max_features
        self.bag, self.out_bag = self._bagging(X, y)

    def _bagging(self, X, y):
        indexs = np.arange(X.shape[0])
        in_bag = np.random.choice(indexs, size=(X.shape[0]))
        out_of_bag = indexs[~np.in1d(indexs, in_bag)]
        return (X[in_bag], y[in_bag]), (X[out_of_bag], y[out_of_bag])

    def _build(self, X: np.ndarray, y: np.ndarray, curr_depth: int = 0) -> Union['DecisionTreeNode', DecisionTreeLeaf]:
        if self.max_depth is not None and (curr_depth >= self.max_depth or curr_depth < 0):
            return DecisionTreeLeaf(y)
        flag_leaf, gain_val, feature, value = True, 0, 0, None
        for i in np.random.choice(X.shape[1], size=self.max_features, replace=False):
            for v in np.unique(X[:, i]):
                l = X[:, i] == 0
                r = X[:, i] != 0
                gain_curr = gain(y[l], y[r], self.criterion)

                if gain_curr >= gain_val and \
                        np.sum(r) >= self.min_samples_leaf and \
                        np.sum(l) >= self.min_samples_leaf:
                    flag_leaf, gain_val, feature, value = False, gain_curr, i, v
                    l_tree, r_tree = l.copy(), r.copy()
        if flag_leaf:
            return DecisionTreeLeaf(y)
        else:
            return DecisionTreeNode(feature, value,
                                    self._build(X[l_tree], y[l_tree], curr_depth + 1),
                                    self._build(X[r_tree], y[r_tree], curr_depth + 1))

    def fit(self) -> NoReturn:
        X, y = self.bag
        self.root = self._build(X, y, 0)

    def _go_to_leaf(self, x, node) -> Union['DecisionTreeNode', DecisionTreeLeaf]:
        if x[node.split_dim] < node.split_value:
            return node.left
        else:
            return node.right

    def _prob(self, x, node) -> Dict[Any, float]:
        if isinstance(node, DecisionTreeLeaf):
            return node.dict
        return self._prob(x, self._go_to_leaf(x, node))

    def _predict_proba(self, X: np.ndarray) ->  List[Dict[Any, float]]:
        return [self._prob(x, self.root) for x in X]
        
    def predict(self, X):
        proba = self._predict_proba(X)
        return np.array([max(p.keys(), key=lambda k: p[k]) for p in proba], dtype=np.int)

    def calc_error(self, X, y):
        return 1 - np.sum(self.predict(X) != y) / y.shape[0]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1,
                 max_features="auto", n_estimators=10):
        if criterion == "gini":
            self.criterion = gini
        else:
            self.criterion = entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        self.trees = []
    
    def fit(self, X, y):
        for _ in np.arange(self.n_estimators):
            tree = DecisionTree(X, y,
                               criterion=self.criterion,
                               max_depth=self.max_depth,
                               min_samples_leaf=self.min_samples_leaf,
                               max_features=self.max_features)
            tree.fit()
            self.trees.append(tree)
    
    def predict(self, X):
        preds = np.zeros((self.n_estimators, X.shape[0]), dtype=int)
        for i, tree in enumerate(self.trees):
            preds[i] = tree.predict(X)
        return np.array([np.bincount(pred).argmax() for pred in preds.T])

    def feature_importance(self):
        n = self.trees[0].out_bag[0].shape[1]
        result = np.zeros((self.n_estimators, n))
        for i, tree in enumerate(self.trees):
            X_ = tree.out_bag[0]
            y =  tree.out_bag[1]
            prec_error = tree.calc_error(X_, y)
            for j in np.arange(n):
                X_curr = X_.copy()
                X_curr[:, j] = X_curr[np.random.permutation(range(X_.shape[0])), j]
                result[i, j] = prec_error - tree.calc_error(X_curr, y)
        return np.mean(result, axis=0)

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc):
    return rfc.feature_importance()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/4065380050.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.max_features = np.floor(np.sqrt(X.shape[1])).astype(np.int)
/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/2566292134.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.2

Accuracy: 1.0
Importance: [9.56501527e-04 3.06754698e-04 1.71396689e-01 1.67437805e-01
 3.14903655e-01 1.52921336e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [8]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [9]:
X, y_age, y_sex, features = read_dataset("vk.csv")
y_age = preprocessing.LabelEncoder().fit_transform(y_age)
y_sex = preprocessing.LabelEncoder().fit_transform(y_sex)
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [10]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/4065380050.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.max_features = np.floor(np.sqrt(X.shape[1])).astype(np.int)
/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/2566292134.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.2

Accuracy: 0.7301387137452712
Most important features:
1. ovsyanochan
2. rhymes
3. 4ch
4. styd.pozor
5. mudakoff
6. bestad
7. dayvinchik
8. leprum
9. rapnewrap
10. iwantyou
11. pixel_stickers
12. pravdashowtop
13. reflexia_our_feelings
14. i_d_t
15. tumblr_vacuum
16. top_screens
17. xfilm
18. girlmeme
19. i_des
20. bot_maxim


#### Пол

In [11]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/4065380050.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.max_features = np.floor(np.sqrt(X.shape[1])).astype(np.int)
/var/folders/j0/1l0qrqy50rb8ph6trtn7qkyr0000gn/T/ipykernel_13453/2566292134.py:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.2

Accuracy: 0.8764186633039092
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. be.beauty
6. 9o_6o_9o
7. thesmolny
8. igm
9. zerofat
10. sh.cook
11. soverwenstvo.decora
12. rapnewrap
13. reflexia_our_feelings
14. rhymes
15. bon
16. femalemem
17. woman.blog
18. combovine
19. be.women
20. h.made


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [12]:
X, y = synthetic_dataset(1000)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent',
    loss_function='MultiClass'
)

rfc.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
)

print("Accuracy:", np.mean(rfc.predict(X_validation).flatten() == y_validation))
print("Importance:", rfc.get_feature_importance())

Accuracy: 1.0
Importance: [9.42673024e-03 7.23075334e-03 2.92434094e+01 2.73289842e+01
 4.34046314e+01 6.31748762e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [13]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

rfc = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=28,
    logging_level='Silent',
    loss_function='MultiClass'
)

#### Возраст

In [14]:
rfc.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval))

print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7452711223203027
Most important features:
1. ovsyanochan
2. 4ch
3. styd.pozor
4. mudakoff
5. rhymes
6. dayvinchik
7. leprum
8. bot_maxim
9. xfilm
10. fuck_humor


#### Пол

In [15]:
rfc.fit(X_train, y_sex_train, eval_set=(X_eval, y_sex_eval))

print("Accuracy:", np.mean(rfc.predict(X_test).flatten() == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(rfc.feature_importances_, features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8726355611601513
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. modnailru
5. igm
6. thesmolny
7. femalemem
8. 9o_6o_9o
9. zerofat
10. cook_good
